## SICP 习题 （2.75）解题总结：消息传递风格的对象构造

SICP 习题 2.75 要求我们按照上文(make-from-real-imag)的代码仿写一个(make-from-mag-ang).

题目的灵魂还是数据的封装，就是不管我们底层怎么表达，上层是可以呈现一致的访问手段的。

就像题目中提到的复数，底层可以保存为 实部 和 虚部， 也可以保存为 幅度 和角度， 但是上层呈现出来的访问方式可以是一样的。

书中已经有(make-from-real-imag)的实现：

In [38]:
; message style implementation:
(define (make-from-real-imag x y)
  (define (dispatch op)
    (cond ((eq? op 'real-part) x)
	  ((eq? op 'imag-part) y)
	  ((eq? op 'magnitude)
	   (sqrt (+ (square x) (square y))))
	  ((eq? op 'angle) (atan (/ y x)))
	  (else 
	   (error "Unknown op -- MAKE-FROM-REAL-IMAG" op))))
  dispatch)

我们只需要仿写就好了， 这两部分代码从一定意义上来讲是反过来的：一个通过实部和虚部计算幅度和角度，另一个通过幅度和角度计算实部和虚部：

In [40]:
(define (make-from-mag-ang x y)
  (define (dispatch op)
    (cond ((eq? op 'real-part) (* x (cos y)))
	  ((eq? op 'imag-part) (* x (sin y)))
	  ((eq? op 'magnitude) x)
	  ((eq? op 'angle) y)
	  (else 
	   (error "Unknown op -- MAKE-FROM-MAG-ANG" op))))
  dispatch)

不过在Calysto Scheme里运行以上代码会遇到问题，就是三角函数在Calysto Scheme里没有实现。

于是我又偷懒通过(python-exec)把python里的三角函数包进来了：

In [41]:
(import 'math)

(python-exec "
def cos(input_number):
    return math.cos(input_number)
             ")

(python-exec "
def sin(input_number):
    return math.sin(input_number)
             ")

(python-exec "
def atan(input_number):
    return math.atan(input_number)
             ")

另外还需要一个求平方的函数(square x)，好像这个函数我们已经实现很多遍了。。。。。

In [42]:
(define (square x) (* x x))

接着是各种复数的运行操作和获取操作：

In [43]:

(define (add-complex z1 z2)
  (make-from-real-imag (+ (real-part z1) (real-part z2))
		       (+ (imag-part z1) (imag-part z2))))

(define (sub-complex z1 z2)
  (make-from-real-imag (- (real-part z1) (real-part z2))
		       (- (imag-part z1) (imag-part z2))))

(define (mul-complex z1 z2)
  (make-from-mag-ang (* (magnitude z1) (magnitude z2))
		     (+ (angle z1) (angle z2))))

(define (div-complex z1 z2)
  (make-from-mag-ang (/ (magnitude z1) (magnitude z2))
		     (- (angle z1 ) (angle z2))))



(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

(define (apply-generic op arg) (arg op))

(define (display-complex complex)
 (display "--------------------") (newline)
  (display "real part:") (display (real-part complex)) (newline)
  (display "imag part:") (display (imag-part complex)) (newline)
  (display "magnitude:") (display (magnitude complex)) (newline)
  (display "angle    :") (display (angle complex))(newline)
  (display "--------------------") (newline)
)


于是我们就多了一种构建复数的方法：

In [44]:
(define complex1 (make-from-mag-ang 2 0.4))

In [45]:
(display-complex complex1)

--------------------
real part:1.8421219880057702
imag part:0.778836684617301
magnitude:2
angle    :0.4
--------------------


In [46]:
(define complex2 (make-from-real-imag 1.8421219880057702 0.778836684617301))

In [47]:
(display-complex complex2)

--------------------
real part:1.8421219880057702
imag part:0.778836684617301
magnitude:2.0
angle    :0.4
--------------------


打包测试一下：

In [48]:
(define (start-test-2-75)
  (define complex1 (make-from-mag-ang 2 0.4))
  (define complex2 (make-from-mag-ang 2 0.4))

  (define result1 (add-complex complex1 complex2))
  (display-complex result1)(newline))

In [49]:
(start-test-2-75)

--------------------
real part:3.6842439760115404
imag part:1.557673369234602
magnitude:4.0
angle    :0.4
--------------------

